# RSI Strategy - Relative Strength Index

This notebook demonstrates how to implement and backtest an RSI (Relative Strength Index) strategy.

**Strategy Logic:**
- Buy when RSI < 30 (oversold)
- Sell when RSI > 70 (overbought)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from typing import Any, Dict, List

from simple_backtest import Backtest, BacktestConfig
from simple_backtest.strategy.strategy_base import Strategy
from simple_backtest.visualization.plotter import plot_equity_curve

## 1. Implement RSI Strategy

In [ ]:
class RSIStrategy(Strategy):
    """RSI momentum strategy."""
    
    def __init__(self, period: int = 14, oversold: float = 30, overbought: float = 70, shares: float = 100):
        """Initialize RSI strategy.
        
        :param period: RSI calculation period
        :param oversold: Oversold threshold (buy signal)
        :param overbought: Overbought threshold (sell signal)
        :param shares: Shares to trade
        """
        super().__init__(name=f"RSI_{period}")
        self.period = period
        self.oversold = oversold
        self.overbought = overbought
        self.shares = shares
    
    def calculate_rsi(self, prices: pd.Series) -> float:
        """Calculate RSI indicator."""
        if len(prices) < self.period + 1:
            return 50.0  # Neutral
        
        # Calculate price changes
        deltas = prices.diff()
        
        # Separate gains and losses
        gains = deltas.where(deltas > 0, 0)
        losses = -deltas.where(deltas < 0, 0)
        
        # Calculate average gains and losses
        avg_gain = gains.tail(self.period).mean()
        avg_loss = losses.tail(self.period).mean()
        
        if avg_loss == 0:
            return 100.0
        
        # Calculate RS and RSI
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        
        return rsi
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Generate signal based on RSI."""
        # Calculate RSI
        rsi = self.calculate_rsi(data['Close'])
        
        # Check if we have open positions
        has_position = False
        if trade_history:
            total_bought = sum(t['shares'] for t in trade_history if t['signal'] == 'buy')
            total_sold = sum(t['shares'] for t in trade_history if t['signal'] == 'sell')
            has_position = total_bought > total_sold
        
        # Generate signal
        if rsi < self.oversold and not has_position:
            return {"signal": "buy", "size": self.shares, "order_ids": None}
        elif rsi > self.overbought and has_position:
            return {"signal": "sell", "size": self.shares, "order_ids": None}
        else:
            return {"signal": "hold", "size": 0, "order_ids": None}

## 2. Download Market Data

In [ ]:
# Download Tesla data
ticker = "TSLA"
start_date = "2020-01-01"
end_date = "2023-12-31"

data = yf.download(ticker, start=start_date, end=end_date)
print(f"Downloaded {len(data)} rows of {ticker} data")
data.head()

## 3. Visualize RSI Indicator

In [ ]:
import matplotlib.pyplot as plt

# Calculate RSI for visualization
def calc_rsi_series(prices, period=14):
    deltas = prices.diff()
    gains = deltas.where(deltas > 0, 0)
    losses = -deltas.where(deltas < 0, 0)
    avg_gains = gains.rolling(window=period).mean()
    avg_losses = losses.rolling(window=period).mean()
    rs = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + rs))
    return rsi

rsi_series = calc_rsi_series(data['Close'])

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Price plot
ax1.plot(data.index, data['Close'], label='Close Price')
ax1.set_ylabel('Price ($)')
ax1.set_title(f'{ticker} Price and RSI Indicator')
ax1.legend()
ax1.grid(True, alpha=0.3)

# RSI plot
ax2.plot(data.index, rsi_series, label='RSI(14)', color='purple')
ax2.axhline(y=70, color='r', linestyle='--', label='Overbought (70)')
ax2.axhline(y=30, color='g', linestyle='--', label='Oversold (30)')
ax2.fill_between(data.index, 30, 70, alpha=0.1, color='gray')
ax2.set_ylabel('RSI')
ax2.set_xlabel('Date')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()

## 4. Run Backtest with Default Parameters

In [ ]:
# Configure backtest
config = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=50,
    commission_type="percentage",
    commission_value=0.001,
    execution_price="open",
    risk_free_rate=0.02,
)

# Create and run RSI strategy
rsi_strategy = RSIStrategy(period=14, oversold=30, overbought=70, shares=50)

backtest = Backtest(data=data, config=config)
results = backtest.run([rsi_strategy])

# Display metrics
print("\n=== RSI Strategy Results ===")
for key, value in results[rsi_strategy.get_name()]['metrics'].items():
    print(f"{key:25s}: {value:>12.2f}")

In [ ]:
# Plot equity curve
fig = plot_equity_curve(results)
fig.show()

## 5. Parameter Optimization

Test different RSI parameters to find optimal settings.

In [ ]:
# Test different RSI periods and thresholds
strategies = []

# Test different periods
for period in [7, 14, 21]:
    strategies.append(RSIStrategy(period=period, oversold=30, overbought=70, shares=50))

# Test different thresholds
strategies.append(RSIStrategy(period=14, oversold=20, overbought=80, shares=50))
strategies.append(RSIStrategy(period=14, oversold=40, overbought=60, shares=50))

# Run all strategies
backtest = Backtest(data=data, config=config)
results = backtest.run(strategies)

# Compare results
comparison_df = pd.DataFrame({
    name: res['metrics'] 
    for name, res in results.items() 
    if name != 'benchmark'
}).T

print("\n=== RSI Parameter Comparison ===")
comparison_df[['total_return', 'sharpe_ratio', 'max_drawdown', 'win_rate', 'total_trades']].sort_values('sharpe_ratio', ascending=False)

In [ ]:
# Plot all equity curves
fig = plot_equity_curve(results)
fig.show()

## 6. Analyze Trade Distribution

In [ ]:
# Get best performing strategy
best_strategy = comparison_df['sharpe_ratio'].idxmax()
trades = results[best_strategy]['trade_history']

# Analyze trades
trades_df = pd.DataFrame(trades)
sell_trades = trades_df[trades_df['signal'] == 'sell']

# Plot P&L distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(sell_trades['pnl'], bins=20, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='r', linestyle='--', label='Break-even')
plt.xlabel('Profit/Loss ($)')
plt.ylabel('Frequency')
plt.title(f'Trade P&L Distribution - {best_strategy}')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
cumulative_pnl = sell_trades['pnl'].cumsum()
plt.plot(cumulative_pnl.values)
plt.xlabel('Trade Number')
plt.ylabel('Cumulative P&L ($)')
plt.title('Cumulative P&L over Trades')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBest Strategy: {best_strategy}")
print(f"Win Rate: {(sell_trades['pnl'] > 0).sum() / len(sell_trades) * 100:.1f}%")
print(f"Average Win: ${sell_trades[sell_trades['pnl'] > 0]['pnl'].mean():.2f}")
print(f"Average Loss: ${sell_trades[sell_trades['pnl'] <= 0]['pnl'].mean():.2f}")